## Setup

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb  5 18:04:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Instalação

In [ ]:
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached keras_nlp-0.3.0-py3-none-any.whl (142 kB)
  Using cached tensorflow_text-2.11.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.8 MB)
  Using cached tensorflow-2.11.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (588.3 MB)
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow_text-2.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 82.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple

### Imports

In [ ]:
import os
import logging

import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Configurações para o fine-tunning

In [ ]:
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 1024
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 512
BATCH_SIZE = 8
LEARNING_RATE = 1e-4
MAX_EPOCHS = 5


MODEL_CHECKPOINT = "t5-base"

## Download do dataset

Será utilizado o dataset [Extreme Summarization (XSum)](https://arxiv.org/abs/1808.08745). Ele possui artigos da BBC juntamente com seus resumos, contendo 226.711 artigos.

A função load_dataset do Hugging Face será utilizada para carregar o dataset.

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("xsum", split="train")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


Features presentes no dataset:

- **document**: o artigo da BBC em si
- **summary**: o resumo do artigo
- **id**: o ID do par artigo-resumo

In [ ]:
print(raw_datasets)

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})


In [ ]:
#printando um exemplo
print(f"Artigo:{raw_datasets[0]['document']}\nResumo:{raw_datasets[0]['summary']}")

Artigo:The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but i

Dividindo os dados em 90% treino e 10% teste

In [ ]:
raw_datasets = raw_datasets.train_test_split(
    train_size=TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

## Pré-processamento dos dados


Como o T5 será o modelo em que o fine-tunning será aplicado, o tokenizador a ser utilizada também é o utilizado no T5. Através do tokenizador, o T5 irá tokenizar o texto com base em seu vocabulário, além de já converter os tokens para seus respectivos id's.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Como o T5 é capaz de executar diversas tasks em um formato text-to-text, aqui é definido o prefixo a ser colocado antes do texto de input, indicando que o T5 deve resumir tal texto.

In [ ]:
prefix = "summarize: "

A função a seguir é responsável por adicionar o prefixos aos textos de entrada, tokenizar os textos de entrada, mas também tokenizar a variável alvo, que são os resumos.

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


Em seguida, o dataset carregado é transformado utilizando a função definida acima.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

## Criando o modelo

Como será uma tarefa textual em que uma sequência é convertida em outra sequência, a classe a ser utilizada para carregar o modelo em que o fine-tunning será aplicado será a TFAutoModelForSeq2SeqLM.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/851M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


O data collator abaixo será utilizado para dividir os dados em batches, além de aplicar padding caso seja necessário.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

Abaixo os dados são divididos em treino, teste e geração. Os dados de geração são utilizados para calcular a métrica ROUGE durante o treinamento.

Além disso, os dados também são divididos em batches.

In [ ]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

Compilando o modelo:

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


## Treino e avaliação do modelo

O modelo será avaliado de acordo com as métricas Rouge-1, Rouge-2 e Rouge-L.

In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

Função que irá calcular as métricas durante o treinamento:

In [ ]:
import keras_nlp
#import rouge_score

#rouge_l = keras_nlp.metrics.RougeL()
rouge = Rouge()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    #result = rouge_l(decoded_labels, decoded_predictions)
    result = rouge.get_scores(decoded_predictions, decoded_labels, avg=True)
    
    result = {"Rouge-1": result["rouge-1"]["f"], "Rouge-2": result["rouge-2"]["f"], "RougeL": result["rouge-l"]["f"]}

    return result


Iniciando o treinamento:

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]


model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

Epoch 1/5
2551/2551 [==============================] - 1168s 458ms/step - loss: 2.2009 - val_loss: 1.9725 - Rouge-1: 0.3032 - Rouge-2: 0.0971 - RougeL: 0.2530
Epoch 2/5
2551/2551 [==============================] - 1160s 455ms/step - loss: 1.9585 - val_loss: 1.9529 - Rouge-1: 0.3004 - Rouge-2: 0.0985 - RougeL: 0.2525
Epoch 3/5
2551/2551 [==============================] - 1162s 456ms/step - loss: 1.7882 - val_loss: 1.9597 - Rouge-1: 0.3015 - Rouge-2: 0.0957 - RougeL: 0.2466
Epoch 4/5
2551/2551 [==============================] - 1159s 454ms/step - loss: 1.6445 - val_loss: 1.9800 - Rouge-1: 0.2935 - Rouge-2: 0.0987 - RougeL: 0.2430
Epoch 5/5
2551/2551 [==============================] - 1157s 454ms/step - loss: 1.5244 - val_loss: 2.0095 - Rouge-1: 0.2911 - Rouge-2: 0.0965 - RougeL: 0.2419


Salvando o modelo treinado:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save_weights('t5_base_summary_weights.h5')

## Inferência

In [ ]:
model.load_weights('/content/drive/MyDrive/t5_base_summary_weights.h5')

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def calculate_tfidf(text, texts):
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(texts)
    feature_names = tfidf.get_feature_names_out()
    tfidf_values = tfidf.transform([text]).toarray()[0]
    tfidf_df = pd.DataFrame({'feature_names': feature_names, 'tfidf_values': tfidf_values})
    hashtags = tfidf_df.nlargest(5, 'tfidf_values').feature_names.to_numpy()
    result = [f"#{h}" for h in hashtags]
    return result


In [ ]:
def generate_summary_news(index):
  resumo = summarizer(
        raw_datasets["test"][index]["document"],
        min_length=MIN_TARGET_LENGTH,
        max_length=128,
      )[0]['summary_text']
  
  hashtags = calculate_tfidf(raw_datasets['test'][index]['document'],raw_datasets['test']['document'])
  print(f"Texto:\n{raw_datasets['test'][index]['document']}\n")
  print(f"Resumo:\n{resumo}\n")
  for item in hashtags:
    print(item, end=' ')


In [ ]:
generate_summary_news(17)

Texto:
The former Manchester United and Wales midfielder is one of 25 European football legends taking part in the ultimate champions match on a floating pitch on 2 June.
The free festival will run alongside the Champions League finals.
It runs from 1-4 June and is expected to attract over 200,000 people.
The women's final will be held on 1 June at Cardiff City Stadium and the men's final on 3 June at the Principality Stadium.
Both kick off at 19:45 BST.
More players for the ultimate champions match are to be announced next week.
Guide to Champions League in Cardiff
Champions League 'city's biggest event'
Champions League 'ring of steel' in city

Resumo:
Cardiff City's biggest football festival is to take place in June.

#champions #june #league #ultimate #cardiff 

In [ ]:
generate_summary_news(11)

Texto:
The 1,901 counterfeit 200-euro banknotes were seized in an operation in the northern city of Oporto, police said in a statement.
A 46-year-old foreign man was arrested, police added, but neither his identity nor his nationality have been revealed.
The counterfeit notes were "of exceptional quality", officials said.
They are thought to be of a type that has been in circulation for a decade, with specimens turning up mostly in Spain, Italy, Germany, Bulgaria and Portugal.
This, police said, is the largest batch found so far.
The BBC's Alison Roberts in Lisbon says it is the second seizure of counterfeit euros this month in Oporto.
Police recently dismantled a suspected ring composed of five Portuguese citizens, accused of counterfeiting and passing off 50 and 20-euro notes.
That operation resulted in the seizure of fake notes worth 30,000 euros.
Portuguese police said the euro's growing importance as an international currency had made it an attractive forging target for organised 

In [ ]:
print(raw_datasets['test'][17]['document'])

The former Manchester United and Wales midfielder is one of 25 European football legends taking part in the ultimate champions match on a floating pitch on 2 June.
The free festival will run alongside the Champions League finals.
It runs from 1-4 June and is expected to attract over 200,000 people.
The women's final will be held on 1 June at Cardiff City Stadium and the men's final on 3 June at the Principality Stadium.
Both kick off at 19:45 BST.
More players for the ultimate champions match are to be announced next week.
Guide to Champions League in Cardiff
Champions League 'city's biggest event'
Champions League 'ring of steel' in city
